# Analyzing a single damped sinusoid

This is the simplest possible example: we analyze a single damped sinusoid in white Gaussian noise. We use this case study to demonstrate some basic features of ringdown, including:

* how to measure generic frequencies and damping rates
* how to set up a fit and run it
* how to make some useful plots

Let's get started!

This first cell must be executed first in order for Stan's multiprocessing code to not crash some MacOS setups; on linux it's not needed.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
from pylab import *
import arviz as az
import seaborn as sns
import pandas as pd
import ringdown as rd
import qnm

sns.set(context='notebook', palette='colorblind')

In [ ]:
def planck_window(t, tMin=0., tMax=1., tol=.005, rolloff=False):
    """
A smooth function f mapping [tMin, tMax] to [0, 1], with
f(tMin)=0, f(tMax)=1, all derivatives -> 0 as t->0 or t->tMax.
f(t) = 1/(1 + exp(z)) where z = 1/t - 1/(1-t) for the case
tMin=0, tMax=1.
tol controls the minimum deviation of t from 0 or tMax, with
a tol of .005 giving abs(z) < ~200, where exp(z) is 1.e86 or 1.e-86

If rolloff, instead has f(0)=1, f(tMax)=0.
    """
    if rolloff:
        return helper.planck_window(-t, tMin=-tMax, tMax=-tMin, tol=tol)

    if tMax==1. and tMin==0.:
        # If t <= .005, z >= 200. - 1./199. and exp(z) > 1.e86 ~ infinity
        # Similar for t > .995 but exp(z) < 1.e-86 ~ 0
        safe = (t>tol)*(t<(1.-tol))
        safeT = safe*t + (1. - safe)*.5 # use t = 0.5 temporarily for unsafe values
        safeZ = 1./safeT - 1./(1. - safeT)
        return safe*1./(1. + np.exp(safeZ)) + (t >= (1.-tol))

    return planck_window((t-tMin)/(tMax-tMin))

In [ ]:
def fac(ffreq,l,m,n,chi):
    ome=qnm.modes_cache(s=-2,l=l,m=m,n=n)(a=chi)[0]
#     return (ffreq-ome)/(ffreq-np.conj(ome))
    return (ffreq-ome)/(ffreq-np.conj(ome))*(ffreq+np.conj(ome))/(ffreq+ome)

## Data

Let's create the simplest possible signal and add Gaussian noise to it. Although `rigndown` contains tools to create and maniuplate more complex ringdown and inspiral-merger-ringdown waveforms, we do not need that for this very simple example.

### Signal

First, create a time array based on a specified sampling rate and duration; center it around a target time $t_0$.

In [ ]:
# define sampling rate and duration (make these powers of two) 
fsamp = 8192
duration = 4
t0 = 0

delta_t = 1/fsamp
tlen = int(round(duration / delta_t))
epoch = t0 - 0.5*tlen*delta_t

time = np.arange(tlen)*delta_t + epoch

We can now use the above timestmps to create a simulated damped sinusoid starting at $t_0$; for good measure, let's extend the signal before $t_0$ to be a corresponding ring-up (the inference is insensitive to what happens before $t_0$, so we can always just let the signal vanish for $t < t_0$, as long as no step in the data conditioning involves taking a Fourier transform---which is true in the example here).

In [ ]:
chi=0.692085186818
M_est=65
t_unit=M_est*2950./2/299792458

In [ ]:
wf_kws = dict(
    A1 = 2,
    phi1 = 0,
    A2 = 16,
    phi2 = np.pi/3*2.,
    f1 = np.real(qnm.modes_cache(s=-2,l=2,m=2,n=0)(a=chi)[0])/t_unit/2/np.pi,
    gamma1 = -np.imag(qnm.modes_cache(s=-2,l=2,m=2,n=0)(a=chi)[0])/t_unit,
    f2 = np.real(qnm.modes_cache(s=-2,l=2,m=2,n=1)(a=chi)[0])/t_unit/2/np.pi,
    gamma2 = -np.imag(qnm.modes_cache(s=-2,l=2,m=2,n=1)(a=chi)[0])/t_unit,
)

def get_signal(time, A1, phi1, f1, gamma1, A2, phi2, f2, gamma2):
    s = A1*cos(2*np.pi*f1*(time-t0) + phi1)*exp(-gamma1*abs(time-t0)) \
        +A2*cos(2*np.pi*f2*(time-t0) + phi2)*exp(-gamma2*abs(time-t0))
    s*=np.heaviside(time,0.5)
    s1=A2*cos(2*np.pi*f2*0.5*(time-t0) + phi2)*(1-np.heaviside(time,0.5))
    window=planck_window(time, tMin=-1.2, tMax=-0.5)
    return rd.Data((s+s1)*window, index=time)

def overtone_alone(time, A1, phi1, f1, gamma1, A2, phi2, f2, gamma2):
    s = 0*cos(2*np.pi*f1*(time-t0) + phi1)*exp(-gamma1*abs(time-t0)) \
        +A2*cos(2*np.pi*f2*(time-t0) + phi2)*exp(-gamma2*abs(time-t0))
    s*=np.heaviside(time,0.5)
    s1=A2*cos(2*np.pi*f2*0.5*(time-t0) + phi2)*(1-np.heaviside(time,0.5))
    window=planck_window(time, tMin=-1.2, tMax=-0.5)
    return (s+s1)*window

def fft_load(time, A1, phi1, f1, gamma1, A2, phi2, f2, gamma2):
    s = A1*cos(2*np.pi*f1*(time-t0) + phi1)*exp(-gamma1*abs(time-t0)) \
        +A2*cos(2*np.pi*f2*(time-t0) + phi2)*exp(-gamma2*abs(time-t0))
    s*=np.heaviside(time,0.5)
    s1=A2*cos(2*np.pi*f2*0.5*(time-t0) + phi2)*(1-np.heaviside(time,0.5))
    window=planck_window(time, tMin=-1.2, tMax=-0.5)
    
    fpsi422=np.fft.ifft((s+s1)*window,norm='ortho')
    ffreq=np.fft.fftfreq(len(s),d=(time[1]-time[0])/t_unit)*2*np.pi
    return ffreq,fpsi422

signal = get_signal(time, **wf_kws)
signal_overtone = overtone_alone(time, **wf_kws)
ffreq,fpsi422 = fft_load(time, **wf_kws)

cond_data=(np.fft.fft(fac(ffreq,2,2,0,chi)*fpsi422,norm='ortho'))

In [ ]:
signal.plot(label='signal')
axvline(t0, ls=':', c='k', label='start time')
plot(time,(cond_data))
xlim(-0.03, 0.03)
xlabel('time (s)')
ylabel('strain')
legend();

In [ ]:
semilogy(time,abs((cond_data)))
semilogy(time,abs(0.5*(signal_overtone)))
axvline(t0+t_unit, ls=':', c='k', label='start time')
xlim(-0.001, 0.006)
ylim(1e-5,10)

### Noise

Let's add some white Gaussian noise to the signal to obtain our data. This means the autocorrelation function (ACF) of the noise will be $\rho(\tau) = \delta(\tau)$.

In [ ]:
rng = np.random.default_rng(12345)
data = signal + rng.normal(0, 0.1, len(signal))

In [ ]:
data.plot(label='data')
signal.plot(label='signal')
axvline(t0, ls=':', c='k', label='start time')
xlim(-0.03, 0.03)
xlabel('time (s)')
ylabel('strain')
legend();

For our ringdown analysis, we will need the ACFs corresponding to the noise, which we could estimate empirically from the data; in this case, however, we know the ACF to be a delta function at the origin (corresponding to standard Gaussian noise).

In [ ]:
acf = rd.AutoCovariance(zeros_like(data), delta_t=data.delta_t)
acf.iloc[0] = 1

Of course, when working with white noise it is unnecessary to work with the ACF: all we have is a variance (in this case, $\sigma^2 = 1$) and the likelihood simplifies drastically. However, in real life we never have white noise, so `ringdown` expects an ACF and cannot currently accomodate a scalar variance.

## Fit

We are now ready to analyze the ringdown! Let's create a `Fit` object and set up the analysis. To do this, we first need to specify the model to be fit and the number (or kind) of modes to be used. In this case, we will be using the `ftau` model, which is parameterized in terms of damped sinusoids with generic frequencies $f_n$, damping rates $\gamma_n = 1/\tau_n$, amplitudes $A_n$, and phases $\phi_n$, defined such that successive modes are ordered in terms of damping rate ($\gamma_n < \gamma_{n+1}$); for this is example, we will only inject and recover a single mode.

In [ ]:
fit = rd.Fit(model='mchi_aligned_ftau', modes=[(1, -2, 2, 2, 1)])
print(fit.valid_model_options)

fit.add_data(data, acf=acf)
fit.filter_data(chi,M_est,0)
fit.set_target(t0+t_unit, duration=0.015)

In [ ]:
wd=fit.analysis_data
plot(time,((cond_data)))
plot(wd[None], label='H1')
xlim(t0+t_unit,t0+0.015)

Finally, let's set some reasonable prior based on our knowledge of the true signal.

In [ ]:
fit.update_prior(A_scale=2, M_min=25.0, M_max=140.0, flat_A=True)
# fit.update_prior(A_scale=2, M_min=35.0, M_max=140.0, cosi_max=-0.99, flat_A=True)
fit.prior_settings

Our fit is ready to go; run it!

<div class="alert alert-info">

Note

The `run()` method will automatically compile the `PyStan` code the first time any fit runs in a given session, which can take a while.  Subsequent runs will use the precompiled code and be much quicker.

</div>

In [ ]:
fit.run()

## Result

In [ ]:
az.summary(fit.result, var_names=['A','phi', 'M', 'chi'])

In [ ]:
az.plot_trace(fit.result, var_names=['A', 'phi'], compact=True);

In [ ]:
mIMR = M_est
chiIMR = chi

df = pd.DataFrame({
    r'$M / M_\odot$': fit.result.posterior.M.values.flatten(),
    r'$\chi$': fit.result.posterior.chi.values.flatten()
})
pg = sns.PairGrid(df, diag_sharey=False)
pg.map_diag(sns.kdeplot);
pg.map_upper(sns.scatterplot);
pg.map_lower(rd.kdeplot_2d_clevels, levels=[0.9, 0.5, 0.1])

pg.axes[0,0].axvline(mIMR, color='k')
pg.axes[0,1].axvline(chiIMR, color='k')
pg.axes[0,1].axhline(mIMR, color='k')
pg.axes[1,0].axhline(chiIMR, color='k')
pg.axes[1,0].axvline(mIMR, color='k')
pg.axes[1,1].axvline(chiIMR, color='k')
# pg.savefig('res.pdf')

In [ ]:
import os
os.system('say "hi"')